In [1]:
from Environment_one_hot import *
from Networks import *

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

import numpy as np
from tqdm import tqdm

In [2]:
LENGTH = 6000
WIDTH = 5000
DIVISION = 50
K = 6
NUM_BASE_STATIONS = 7

env = Environment(LENGTH, WIDTH, DIVISION, 1, 0, K)
eps = 10e-6

Location and directions of cells are: 
{1: [0, 0, 0], 2: [0, 0, 1], 3: [0, 0, 2], 4: [1500.0, 0, 0], 5: [1500.0, 0, 1], 6: [1500.0, 0, 2], 7: [750.0, 1250.0, 0], 8: [750.0, 1250.0, 1], 9: [750.0, 1250.0, 2], 10: [-750.0, 1250.0, 0], 11: [-750.0, 1250.0, 1], 12: [-750.0, 1250.0, 2], 13: [-1500.0, 0, 0], 14: [-1500.0, 0, 1], 15: [-1500.0, 0, 2], 16: [-750.0, -1250.0, 0], 17: [-750.0, -1250.0, 1], 18: [-750.0, -1250.0, 2], 19: [750.0, -1250.0, 0], 20: [750.0, -1250.0, 1], 21: [750.0, -1250.0, 2]} 
 

Strongest cells for sector (0, 0)
[[5.0, -69.68031402659005], [13.0, -71.07287859388722], [10.0, -77.36490164108055], [16.0, -77.39288922194086], [9.0, -77.7794686317523], [20.0, -79.71838870710408]]


Strongest cells for sector (0, 100)
[[13.0, -68.54875066828438], [5.0, -72.23390665442798], [2.0, -76.24066816535561], [17.0, -81.04742797484525], [20.0, -81.67563656386488], [10.0, -83.59618041497248]]


Strongest cells for sector (50, 0)
[[5.0, -64.51934435908518], [13.0, -75.59831577353562],

In [3]:
class Agent:
    def __init__(self):
        self.loss_fn = tf.keras.losses.huber
        self.actor_optimizer = tf.keras.optimizers.Adam(lr = 0.0001)
        self.critic_optimizer = tf.keras.optimizers.Adam(lr = 0.0002)
        self.actor = ACTOR_NET(K, NUM_BASE_STATIONS)
        self.critic = CRITIC_NET(K, NUM_BASE_STATIONS)
        self.gamma = 0.3

    def learn(self, WRSRP, WHO):
        running_reward = 0
        running_handovers = 0
        max_reward = float('-inf')
        for epoch in tqdm(range(100000)):
            action_probs_history = []
            critic_value_history = []
            rewards_history = []
            
            episode_reward = 0
            src,dest = env.give_src_dest()
            route = env.compute_route(src, dest)
            state = route.popleft()
            depth = 3*NUM_BASE_STATIONS
            one_hot_cell = make_one_hot(env.sector_cells[src][0][0], depth)
            one_hot_direction = make_one_hot(state[-1]+1, 8)
            state = state[:-1]
            state.extend(one_hot_direction) #According route choosing one of the 8 directions
            state.extend(one_hot_cell) #Setting strongest cell as the initial serving cell (one_hot)
            done = False
            handovers = 0
            
            with tf.GradientTape(persistent = True) as tape:
                while not done:
                    norm_state = list(state[:])
                    norm_state[0] = norm_state[0]/(LENGTH//2)
                    norm_state[1] = norm_state[1]/(WIDTH//2) 
                    norm_state = np.array(norm_state)[np.newaxis]
                    action_probs, critic_value = self.actor.model(norm_state), self.critic.model(norm_state)
                    critic_value_history.append(critic_value[0, 0])
                    
                    action = np.random.choice(K, p=np.squeeze(action_probs))
                    action_probs_history.append(tf.math.log(action_probs[0, action]))
                    
                    next_state, reward, done, change = env.step(state, route, action, dest)
                    
                    state = next_state
                    if change:
                        handovers+=1
                    reward*=WRSRP
                    reward-=change*WHO
                    rewards_history.append(reward)
                    episode_reward += reward
                    
                returns = []
                discounted_sum = 0
                
                for r in rewards_history[::-1]:
                    discounted_sum = r + self.gamma * discounted_sum
                    returns.append(discounted_sum)
                    
                returns.reverse()
                
                returns = np.array(returns)
                returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
                returns = returns.tolist()
                
                if epoch:
                    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
                    running_handovers = 0.05*handovers + (1-0.05)*running_handovers
                else:
                    running_reward = episode_reward
                    running_handovers = handovers
                    
                history = zip(action_probs_history, critic_value_history, returns)
                actor_losses = []
                critic_losses = []
                for log_prob, value, ret in history:
                    diff = ret - value
                    actor_losses.append(-log_prob * diff)
                    critic_losses.append(self.loss_fn(tf.expand_dims(value, 0), tf.expand_dims(ret, 0)))
                actor_loss_value = sum(actor_losses)
                critic_loss_value = sum(critic_losses)
                
            grads = tape.gradient(actor_loss_value, self.actor.model.trainable_variables)
            self.actor_optimizer.apply_gradients(zip(grads, self.actor.model.trainable_variables))
            
            grads = tape.gradient(critic_loss_value, self.critic.model.trainable_variables)
            self.critic_optimizer.apply_gradients(zip(grads, self.critic.model.trainable_variables))
            
            action_probs_history.clear()
            critic_value_history.clear()
            rewards_history.clear()
            
            if epoch and epoch%1000 == 0:
                print(running_reward)
                print(running_handovers)
                if running_reward>=max_reward:
                    max_reward = running_reward
                    tf.keras.models.save_model(self.actor.model, f'model_{WRSRP}_{WHO}.h5')

In [4]:
agent = Agent()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 31)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                1024      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32)               

In [ ]:
agent.learn(1,1)

  1%|▋                                                                        | 1001/100000 [10:51<20:30:27,  1.34it/s]

2.7678878864705796
28.62145893554227


  2%|█▍                                                                       | 2001/100000 [23:03<21:40:48,  1.26it/s]

2.6764856894616416
23.720883905026152


  3%|██▏                                                                      | 3001/100000 [34:24<14:48:52,  1.82it/s]

...

 37%|█████████████████████████▉                                            | 37001/100000 [6:42:27<12:25:18,  1.41it/s]

4.611538186109028
27.497380416090053


 38%|██████████████████████████▉                                            | 38001/100000 [6:53:30<9:54:18,  1.74it/s]

2.6776906481702025
24.96668536669774


 39%|███████████████████████████▏                                          | 38829/100000 [7:02:42<11:05:56,  1.53it/s]
